In [1]:
%load_ext autoreload
%autoreload 2
import sqlite3
import operator
import pandas as pd
import collections
import networkx as nx
import json
import math
import numpy as np
from multiprocessing import Pool
import time
utils = __import__('utils.utils', fromlist=['object'])
train = __import__('utils.train', fromlist=['object'])

In [2]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_100+_train.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(max) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

INTERVAL = 60
TIME_PERIOD = 1
SCALE_METERS = 15000
scaleMeters = SCALE_METERS * 0.2
SIGMA_V = scaleMeters / 100
IMPOSSIBLE = 1e-10

In [3]:
id_to_pos, pos_to_id = utils.get_position(df_agents)

In [4]:
print(df_contacts_tmp.shape[0], df_contacts.shape[0])

232194 72620


In [5]:
prob_agent_id, prob_dir, prob_move = utils.get_agent_next_prob(df_walkers, df_contacts)
prob_pos = utils.map_prob_to_pos(prob_agent_id, id_to_pos)

In [6]:
"""
# agent_id -> next_agent_id
sparse data
"""
# print(collections.Counter([i for dic in prob_agent_id.values() for _, i in dic.items()]))

'\n# agent_id -> next_agent_id\nsparse data\n'

In [7]:
def get_last_2_time(df, walker_id):
    time = sorted(set(df.loc[df['walker_id'] == walker_id]['time'].tolist()))
    if len(time) < 2:
        return None, None
    time1, time2 = time[-1], time[-2]
    return time1, time2

In [8]:

DEBUG = True
"""
Map agent id to its most possible next agent.
"""
link_list = {}
visited_walker_ids = set()

def train(walker_id1):
    posX, posY, negX, negY = [], [], [], []
    time1, time2 = get_last_2_time(df_contacts, walker_id1)
    if not time1 or not time2:
        return #continue
    
    start_time = time.time()
    # in case there're multiple agents tracking the walker
    last_rows = df_contacts.loc[(df_contacts['walker_id'] == walker_id1) & (df_contacts['time'] == time1)]
    pre_rows = df_contacts.loc[(df_contacts['walker_id'] == walker_id1) & (df_contacts['time'] == time2)]
    if last_rows.shape[0] == 0 or pre_rows.shape[0] == 0:
        return #continue
        
    if DEBUG: print('walker_id1: {}, # last_rows: {}, # pre_rows: {}'.format(walker_id1, last_rows.shape, pre_rows.shape))
        
    # among all the tracked records, consider mean velocity as real velocity
    vx, vy = utils.get_mean_v(last_rows, pre_rows)
    avg_dis = math.sqrt(vx ** 2 + vy ** 2)
    if DEBUG: print('velocity: ', time.time() - start_time)
    
    start_time = time.time()
    # get candidate points by time
    candidates = df_contacts.loc[(pd.to_numeric(df_contacts["time"]) < (int(time1) + 100)) & 
                                 (pd.to_numeric(df_contacts["time"]) > (int(time1))) & 
                                 (df_contacts['walker_id'] != walker_id1)]
    next_time = candidates['time'].min()
    candidates = candidates.loc[candidates['time'] == next_time]
    # idx = groupby(['walker_id'])['time'].transform(lambda x: pd.to_numeric(x).min()) == pd.to_numeric(candidates['time'])
    # candidates = candidates[idx]
    if DEBUG: print(f'{candidates.shape}: ', time.time() - start_time)
    
    if candidates.shape[0] == 0:
        return #continue
    
    candidate_agent_probs = {}
    for _, row1 in last_rows.iterrows():
        start_time = time.time()
        
        agent_id1, time1 = row1['agent_id'], row1['time']
        json1 = json.loads(row1['json'])['agentPos']
    
        for _, row2 in candidates.iterrows():
            res = ''
            agent_id2, walker_id2, time2 = row2['agent_id'], row2['walker_id'], row2['time']
            json2 = json.loads(row2['json'])['agentPos']
            
            isPos, thisX = True, []
            if df_walks.loc[df_walks['walker_id'] == walker_id1].iloc[0]['real_id'] == df_walks.loc[df_walks['walker_id'] == walker_id2].iloc[0]['real_id']:
                isPos = True
            else:
                isPos = False
            
            new_prob = 0
            
            # direct
            this_prob = utils.get_direct_prob(prob_agent_id, agent_id1, agent_id2)
            new_prob += this_prob
            res += 'direct: ' + str(this_prob)
            thisX.append(this_prob)

            # distance
            timec1, timec2 = get_last_2_time(df_contacts, walker_id2)
            if not timec1 or not timec2:
                vx2, vy2 = vx, vy
            else:
                last_rows2 = df_contacts.loc[(df_contacts['walker_id'] == walker_id2) & (df_contacts['time'] == timec1)]
                pre_rows2 = df_contacts.loc[(df_contacts['walker_id'] == walker_id2) & (df_contacts['time'] == timec2)]
                if last_rows2.shape[0] == 0 or pre_rows2.shape[0] == 0:
                    vx2, vy2 = vx, vy
                else:
                    vx2, vy2 = utils.get_mean_v(last_rows2, pre_rows2)
            avg_dis2 = math.sqrt(vx2 ** 2 + vy2 ** 2)
            
            this_prob = utils.get_dis_prob(json1, json2, (vx+vx2)/2, (vy+vy2)/2, prob_move[agent_id1] if agent_id1 in prob_move else None,
                                           avg_dis=(avg_dis+avg_dis2)/2, time=int(row2['time']) - int(row1['time']))
            if this_prob <= 1e-10:
                continue
            new_prob += this_prob
            thisX.append(this_prob)
            res += ', distance: ' + str(this_prob)

            # direction
            this_prob = utils.get_direction_prob(json1, json2, prob_dir, agent_id1)
            thisX.append(this_prob)
            new_prob += this_prob
            res += ', directions: ' + str(this_prob)
                
            if DEBUG: print(res)
                
            if isPos:
                posX.append(thisX)
                posY.append(1)
            else:
                negX.append(thisX)
                negY.append(0)
        if DEBUG: print(f'candidates', time.time() - start_time)
        return posX, posY, negX, negY
#     if i % 100 == 99: print(f'{i+1}th/{len(walker_ids)} iter: {time.time() - start_time}')

In [9]:
p = Pool(10)
walker_ids = set(df_contacts['walker_id'].tolist())
print(len(walker_ids))

7677
walker_id1: TLGamCgbnYKupLFBVaWR, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2170259952545166
walker_id1: txDuAjkblSSFisgxXEkZ, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: SbSWZnuqRgWejjKBMiQS, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2823069095611572
walker_id1: aLQHwIJgfRvsYlplaHzR, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2915680408477783
velocity:  0.31132984161376953
walker_id1: VrVRIOncelcUGTItBrvO, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: GBsnNMYiuMYnnvnRbPPr, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.32073211669921875
velocity:  0.3305661678314209
walker_id1: YUjXLDcexgFwtcxYPcff, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: JfreMoOKvTGwIOlkhhnc, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.38162803649902344
velocity:  0.37682199478149414
walker_id1: GDeCoOoIsRiKKuYTRFmK, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: rLFvmeBuqswqvFPQGfpx, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  

walker_id1: yvldZfbKcPlpWLSEVoQv, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2925288677215576
walker_id1: yOqmffifAfJiTVxPQqrY, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.26261115074157715
walker_id1: wnASuaILFjaxzQAMgUTv, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2619669437408447
walker_id1: ZTeqdGQzQfKLSGdQnNhX, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.28013110160827637
(1, 7):  0.9251761436462402
candidates 1.4312713146209717
(4, 7):  0.7965900897979736
candidates 0.36515212059020996
(1, 7):  0.8562850952148438
walker_id1: FJoEySTiAdbeHShoHtcK, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.3271758556365967
(5, 7):  0.9120707511901855
(3, 7):  0.8970801830291748
candidates 2.7670609951019287
(5, 7):  0.853302001953125
walker_id1: tJdtrwLYGFoGSuoLmKSM, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.3193049430847168
(2, 7):  0.9264509677886963
candidates 0.5534498691558838
walker_id1: ZvpGCOTvywCcpVnLFsrn, # last_rows: (1, 7), # p

velocity:  0.21538901329040527
(7, 7):  0.8288938999176025
(1, 7):  0.8438918590545654
candidates 0.876572847366333
candidates 0.8659811019897461
walker_id1: gZWufRuHLrLGkBTRTspu, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.23658108711242676
(3, 7):  0.8660149574279785
(1, 7):  0.8297150135040283
direct: 0.9176470588235294, distance: 100, directions: 0
candidates 0.44388508796691895
walker_id1: gkteqlbIwPeApmBZyAqg, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.29288411140441895
walker_id1: RlHsEbGkiKhFReqoqhwp, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.24207210540771484
(1, 7):  0.8533480167388916
walker_id1: kyfmthKbCCJSptyrVdaF, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.3011021614074707
candidates 0.9752757549285889
candidates 0.5035707950592041
(2, 7):  0.9374549388885498
walker_id1: gCrtXPSVHbVsboibEtGR, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.3073861598968506
candidates 0.511523962020874
walker_id1: tixKvRPEptwNmQKzXpZV, # last_r

candidates 0.39948582649230957
candidates 1.3814020156860352
(1, 7):  0.8055338859558105
candidates 1.8287019729614258
candidates 0.4459109306335449
(1, 7):  0.8620469570159912
walker_id1: wvSyRmHHhbxAQyQYnffq, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: oNJLfxWQEjiNIeuOsGYB, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2110891342163086
velocity:  0.23387813568115234
candidates 1.3120849132537842
walker_id1: wCDsItumYYmQtEvLSlwL, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: fIajIuPhNZsDJzcUUWhJ, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2708737850189209
velocity:  0.26319217681884766
candidates 0.48688578605651855
candidates 0.4246361255645752
walker_id1: MfCxXeFCrTPFptaZChdc, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.1931471824645996
walker_id1: YuYpFDszBddeDSwppjus, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.22975993156433105
walker_id1: FdwEgmxPVcoNSevppGWz, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.25899720191955566


walker_id1: QzhNdmozKGILLTsMNvvR, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.24353790283203125
walker_id1: VFDtnFWvlskacowFKLoj, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2098219394683838
candidates 0.3953220844268799
walker_id1: xJyQAXlVTRqRPOcLZNou, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2561171054840088
candidates 1.0896790027618408
(3, 7):  0.6531009674072266
(7, 7):  0.7492988109588623
candidates 0.8822429180145264
walker_id1: njheDFMhZkUebYzjSZVv, # last_rows: (1, 7), # pre_rows: (1, 7)
(1, 7):  0.7873568534851074
velocity:  0.22906494140625
walker_id1: BfYwrXmMRdXFiDuqLDLB, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.3151369094848633
(2, 7):  0.8008198738098145
walker_id1: dADsdTlFITjEUsMdAWpD, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2699291706085205
candidates 0.38216614723205566
(4, 7):  0.8183372020721436
(4, 7):  0.7607731819152832
(5, 7):  0.7976970672607422
direct: 0.14516129032258066, distance: 100, directions: 0
wa

...

candidates 0.8119540214538574
candidates 0.46276092529296875
candidates 0.4028899669647217
candidates 0.49494004249572754
candidates 0.8176479339599609
candidates 0.36840295791625977
candidates 2.3857107162475586
walker_id1: ZstfwcBrGYKCqqFXgacc, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2005167007446289
walker_id1: wTmGBomScrlipewEPenH, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.15976381301879883
walker_id1: tYWEAyNuHBcZVBBcJvji, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: CooPJaOQbSVCqtKRvxGe, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.22653603553771973
velocity:  0.2105569839477539
candidates 0.9109859466552734
walker_id1: AFBFBztgWMzJKkNDTdxH, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.22699284553527832
walker_id1: zGExCGFsppjZxQtgNRmH, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2104048728942871
walker_id1: cjArvJjvfXhLyrIwyVLs, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2409060001373291
(3, 7):  0.705393075942993

direct: 0.8707386363636364, distance: 100, directions: 0
candidates 0.3619987964630127
candidates 0.7458088397979736
(3, 7):  0.7006049156188965
walker_id1: eMrZKJQwEkXBXiDZiPqn, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.21660494804382324
walker_id1: jnLjAtzlCofSIeYkajuB, # last_rows: (1, 7), # pre_rows: (1, 7)
(5, 7):  0.8079159259796143
velocity:  0.29912710189819336
(1, 7):  0.7724881172180176
(1, 7):  0.871513843536377
(1, 7):  0.7801837921142578
candidates 0.8281691074371338
(2, 7):  0.7716929912567139
(1, 7):  0.7828290462493896
candidates 0.30739283561706543
candidates 0.35301995277404785
walker_id1: lpdIrNpuzWLiSxVDbnuV, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.23079299926757812
candidates 0.365462064743042
walker_id1: yCTcVtrMCdjlshGtWfpn, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.15330195426940918
(4, 7):  0.7131950855255127
(1, 7):  0.6492040157318115
walker_id1: EsqzNZVrDaTwBBmGvmox, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.2324

candidates 0.9906117916107178
walker_id1: HGVpcsIviYRFoFleLGkY, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.10659027099609375
candidates 1.21421480178833
candidates 0.5200042724609375
(2, 7):  0.41733217239379883
walker_id1: NJdnrhoBhEyyTKRklMsD, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.12812280654907227
walker_id1: SdxWZQotdrmjvjiSFWVD, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: KYJLHiwOVwCxcSajwzyv, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.16779494285583496
velocity:  0.15557312965393066
(2, 7):  0.4598519802093506
(4, 7):  0.45318102836608887
candidates 0.4897301197052002
(1, 7):  0.4497549533843994
(1, 7):  0.43831920623779297
walker_id1: RjNBFrdAvCqytTYaQEaa, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.12140393257141113
candidates 0.45142078399658203
candidates 0.24143099784851074
candidates 0.2915198802947998
walker_id1: KroGULXafsZuwZgfPsGW, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.17946600914001465
walker_id1: ZWDas

(5, 7):  0.2728230953216553
candidates 0.1485888957977295
walker_id1: KsHjJExPyVMSycYPEsGQ, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.09007596969604492
walker_id1: mwyfgKNhRlfVaumOrfly, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.08840584754943848
candidates 13.92050313949585
(3, 7):  0.25528693199157715
(2, 7):  0.22089099884033203
walker_id1: PKuqLDzAARHqnBweuFRg, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.08849811553955078
candidates 0.2589578628540039
(1, 7):  0.23654890060424805
candidates 0.683804988861084
walker_id1: ndauKqAvDQOOViDFNdie, # last_rows: (1, 7), # pre_rows: (1, 7)
walker_id1: yzaPgPaRVqJmVUdEAXew, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.09007525444030762
candidates 0.4128859043121338
velocity:  0.07737588882446289
candidates 0.18295812606811523
walker_id1: BOBZKGSBrSVKViWSMrWj, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.10697388648986816
walker_id1: opfKGFDLHPZxBOFVLVdD, # last_rows: (1, 7), # pre_rows: (1, 7)
ve

walker_id1: UFUFlAMZJsHADzfOcpsY, # last_rows: (1, 7), # pre_rows: (1, 7)
velocity:  0.04044198989868164
(2, 7):  0.11224913597106934
candidates 0.1362161636352539


In [10]:
results = p.map(train, walker_ids)

In [11]:
results

[([], [], [], []),
 ([], [], [], []),
 ([[0.13636363636363635, 0.0019333333333357552, 0]], [1], [], []),
 None,
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([[0.06722689075630252, 0.0024776372073787873, 0]], [1], [], []),
 ([], [], [], []),
 None,
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([[0.9128878281622912, 100, 0]], [1], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([[0.0, 100, 0]], [1], [], []),
 ([[0.9128878281622912, 100, 0]], [1], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([[0.9128878281622912, 100, 0]], [1], [], []),
 ([], [], [], []),
 ([], [], [], []),
 None,
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([], [], [], []),
 ([

In [12]:
posX, posY, negX, negY = [], [], [], []
for tmp in results:
    if tmp is None:
        continue
    tposX, tposY, tnegX, tnegY = tmp
    posX += tposX
    posY += tposY
    negX += tnegX
    negY += tnegY

In [13]:
print(np.array(posY).shape, np.array(negY).shape)

(421,) (217,)


In [14]:
# choices = np.random.choice(np.array(posY).shape[0], np.array(negY).shape[0], replace=False)
# choices = np.random.choice(np.array(negY).shape[0], np.array(posY).shape[0], replace=False)

In [15]:
# npX = np.append(np.array(negX), np.take(np.array(posX), choices, 0), axis=0)
# npY = np.append(np.array(negY), np.take(np.array(posY), choices, 0))

# npX = np.append(np.array(posX), np.take(np.array(negX), choices, 0), axis=0)
# npY = np.append(np.array(posY), np.take(np.array(negY), choices, 0))

npX = np.append(np.array(posX), np.array(negX), axis=0)
npY = np.append(np.array(posY), np.array(negY))

In [16]:
np.set_printoptions(threshold=np.inf)

In [17]:
# with open('X.txt', 'w') as f:
#     for item in npX:
#         f.write("%s\n" % item)
# with open('Y.txt', 'w') as f:
#     for item in npY:
#         f.write("%s\n" % item)

In [18]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=500).fit(npX, npY)
clf.predict(npX[:2, :])
clf.predict_proba(npX[:2, :])
clf.score(npX, npY)

0.6504702194357367

## Predicating

In [29]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_100-_test.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_contacts = pd.read_sql_query("SELECT * FROM contacts", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)

In [30]:
"""
Get counter of agent_id Key -> agent_id Values
"""
id_to_pos, pos_to_id = utils.get_position(df_agents)
prob_agent_id = utils.map_prob_to_agent_id(prob_pos, pos_to_id)

In [31]:
"""
# agent_id -> next_agent_id
sparse data
"""
print(collections.Counter([i for dic in prob_agent_id.values() for _, i in dic.items()]))

Counter({1: 1418, 2: 546, 3: 309, 4: 167, 5: 142, 6: 91, 7: 71, 8: 66, 10: 38, 9: 33, 12: 25, 13: 24, 11: 20, 16: 16, 15: 15, 14: 15, 17: 14, 18: 12, 21: 11, 25: 10, 19: 9, 28: 9, 30: 7, 22: 7, 33: 6, 23: 6, 38: 5, 31: 4, 26: 4, 42: 4, 24: 4, 50: 3, 35: 3, 20: 3, 34: 3, 47: 3, 60: 2, 29: 2, 27: 2, 32: 2, 41: 2, 49: 2, 54: 2, 43: 2, 64: 2, 40: 2, 66: 2, 204: 1, 48: 1, 96: 1, 56: 1, 109: 1, 197: 1, 71: 1, 72: 1, 44: 1, 45: 1, 765: 1, 176: 1, 88: 1, 67: 1, 139: 1, 62: 1, 39: 1, 82: 1, 76: 1, 55: 1, 69: 1, 51: 1, 613: 1, 102: 1, 37: 1, 61: 1, 78: 1})


In [32]:
DEBUG = False
"""
Map agent id to its most possible next agent.
"""
link_list = {}
walker_ids = set(df_contacts['walker_id'].tolist())
visited_walker_ids = set()
X, Y = [], []
for walker_id1 in walker_ids:
    time1, time2 = get_last_2_time(df_contacts, walker_id1)
    if not time1 or not time2:
        continue
    
    # in case there're multiple agents tracking the walker
    last_rows = df_contacts.loc[(df_contacts['walker_id'] == walker_id1) & (df_contacts['time'] == time1)]
    pre_rows = df_contacts.loc[(df_contacts['walker_id'] == walker_id1) & (df_contacts['time'] == time2)]
    if last_rows.shape[0] == 0 or pre_rows.shape[0] == 0:
        continue
        
    if DEBUG: print('walker_id1: {}, time: {}, # last_rows: {}, # pre_rows: {}'.format(walker_id1, time, last_rows.shape, pre_rows.shape))
        
    # among all the tracked records, consider mean velocity as real velocity
    vx, vy = utils.get_mean_v(last_rows, pre_rows)
    avg_dis = math.sqrt(vx ** 2 + vy ** 2)
    
    # get candidate points by time
    candidates = df_contacts.loc[(pd.to_numeric(df_contacts["time"]) < (int(time1) + 100)) & (pd.to_numeric(df_contacts["time"]) > (int(time1))) 
                                 & (df_contacts['walker_id'] != walker_id1)]
    idx = candidates.groupby(['walker_id'])['time'].transform(lambda x: pd.to_numeric(x).min()) == pd.to_numeric(candidates['time'])
    candidates = candidates[idx]
    
    if candidates.shape[0] == 0:
        continue
    
    candidate_agent_probs = {}
    for _, row1 in last_rows.iterrows():
        agent_id1, time1 = row1['agent_id'], row1['time']
        json1 = json.loads(row1['json'])['agentPos']
    
        for _, row2 in candidates.iterrows():
            res = ''
            
            agent_id2, walker_id2 = row2['agent_id'], row2['walker_id']
            json2 = json.loads(row2['json'])['agentPos']
            
            new_prob = 0
            X.append([])
            
            # direct
            this_prob = utils.get_direct_prob(prob_agent_id, agent_id1, agent_id2)
            new_prob += this_prob
            res += 'direct: ' + str(this_prob)
            X[-1].append(this_prob)

            # distance
            timec1, timec2 = get_last_2_time(df_contacts, walker_id2)
            if not timec1 or not timec2:
                vx2, vy2 = vx, vy
            else:
                last_rows2 = df_contacts.loc[(df_contacts['walker_id'] == walker_id2) & (df_contacts['time'] == timec1)]
                pre_rows2 = df_contacts.loc[(df_contacts['walker_id'] == walker_id2) & (df_contacts['time'] == timec2)]
                if last_rows2.shape[0] == 0 or pre_rows2.shape[0] == 0:
                    vx2, vy2 = vx, vy
                else:
                    vx2, vy2 = utils.get_mean_v(last_rows2, pre_rows2)
            avg_dis2 = math.sqrt(((vx+vx2)/2) ** 2 + ((vy+vy2)/2) ** 2)

            this_prob = utils.get_dis_prob(json1, json2, (vx+vx2)/2, (vy+vy2)/2, prob_move[agent_id1] if agent_id1 in prob_move else None,
                                           avg_dis=avg_dis2, time=int(row2['time']) - int(row1['time']))

            if this_prob <= IMPOSSIBLE:
                continue
            else:
                new_prob += this_prob
                X[-1].append(this_prob)

            res += ', distance: ' + str(this_prob)

            # direction
            this_prob = utils.get_direction_prob(json1, json2, prob_dir, agent_id1)
            X[-1].append(this_prob)
            new_prob += this_prob
            res += ', directions: ' + str(this_prob)
                
            if DEBUG: print(res)
                
            # a walker's probability to be the next step is the average probability
            weight = json.loads(row2['json'])['distance']
            prob_y = clf.predict_proba(np.array([X[-1]])[:1, :])[0][1]
#             print(clf.predict_proba(npX[:1, :]))
            candidate_agent_probs[walker_id2] = utils.get_avg_prob(candidate_agent_probs, walker_id2, prob_y, weight)
            
    if DEBUG: print(candidate_agent_probs)
    
#     for k in candidate_agent_probs.keys():
#         if k in visited_walker_ids:
#             candidate_agent_probs[k] = (0,0)
    correct_walker_id = max(candidate_agent_probs.items(), key=operator.itemgetter(1))[0] if candidate_agent_probs.items() else 0
    if correct_walker_id in visited_walker_ids:
        continue
    link_list[walker_id1] = correct_walker_id
    visited_walker_ids.add(correct_walker_id)

In [33]:
"""
Connect each linked path into a single path
"""
graph = nx.Graph([(i, j) for i, j in link_list.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)
print(len(connected_components))
print(connected_components)

tried = 0
df_walks_connected = pd.DataFrame(df_walks)
for i, component in enumerate(connected_components):
    new_id = i
    time_set = set()
    for c in component:
        rows = df_walks_connected.loc[df_walks_connected['walker_id'] == c]
        
#         cyclic = False
#         for time in rows['walk_time'].tolist():
#             time_set.add(time)
            
        tried += df_walks_connected.loc[df_walks_connected['walker_id'] == c].shape[0]
        df_walks_connected = df_walks_connected.replace({'walker_id': {c: str(new_id) + 'tmp'}})
        

385
[{'GUmzXaUabfxuiVCSoKNy', 'usdDkJDHldthTDvqDWUj', 'wUwWCNLtHiDpLCmCEmUV', 'mvAyZOazkvdJzSbcmxFS', 'wxKZeRojksLNJSYbQxzt', 'rGoQXmaiIfgcWHmZXTyX', 'meIKKVomjzCoKlzSRZmB', 'oKBrFqaovkAqasNhALNt', 'pczKQNwCQdhzOFRryfmQ', 'RZmXszQbFUJuUMmRgZBJ', 'lDEHuwrqqUiEGczCfjGG', 'cFkUkAzHFXqgRgLhpHeN', 'vfWXUvWDDDqVqInbKTbn', 'kXAvSOAvvTLQJXTnODSy', 'SyVvazuSkwshhhnPYfkX', 'GLfKdPyuinqzHmicfXdr', 'gskvLfDdaVJkEtzWnDLv', 'rFXLoxtWBFyOCmnZbIuW', 'YGtkbPVTmixBIIjByqlJ', 'KqonxKYLsIBfSRQVwLHp'}, {'TSxyNyMglkZXDsNpNSBj', 'tTDqGAvRQCEnTXOGLZuq', 'gUNvFIXqmCWtLRznXowS', 'ZeoxxcMzpLFyPfYOBWGl', 'CpckVAesSQNiRLBtxocj', 'teNWaeQwsQgkqXZzBgSR', 'QrUZrQiIDLjuDOZvtAQP', 'UbmurUOOtdlBGknUMRvR', 'OUrrIVdDGKrsVqMllgnK', 'HkeCfMaTObSIqLyVPEzg', 'vqlCttJhxkDIljrYpuJV', 'VccMZVkFuFjCXojNrMat', 'NeqnkkRweoynTbLCffjm', 'GDGRVnpAtNgCUKFIJrRO', 'vdCzHFcZhZcIutfpjIft', 'XxaCttVOaszsnxQzSKgJ', 'dzzpBEcqjVpjdxQQkuEV', 'hLdYcnfeMgUCAGrkIspt', 'WDsqFoJfFwLhjfEcoCcF'}, {'DKtvRoqEJKIVtEtDsosC', 'ffnyqpnNjJmOMvnJePXW', 'hVGHz

In [34]:
"""
How much percentage of the grouped ids are really from one group
"""

DEBUG = True
walker_ids = set(df_walks_connected['walker_id'].tolist())
correct, total = 0, 0
if DEBUG: print(walker_ids, len(walker_ids))
    
for walker_id in walker_ids:
    id_list = df_walks_connected.loc[df_walks_connected['walker_id'] == walker_id].sort_values('walk_time')['real_id'].tolist()
    dic = collections.Counter(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum(dic.values())

# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')

{'66tmp', '305tmp', 'KcGjhBtZJhwwkNBCsfbS', 'gkbgVwdRyAuPdpAanXPu', 'dBajyZdeczGQepAIJApo', 'XueEndfUJQeTRtGxczPO', 'fbzMBIFDXgawrsddqCtv', '251tmp', 'PUEaZXfprCatkqwxwMKG', '32tmp', '274tmp', 'bBZgtFvHvSltgszciBVd', 'QUgLFZbdBaXGUvNtJpnN', '196tmp', 'UGheQxnMyPUvlfgGoOMC', 'VPubhgbvMLSJjAcbXsoh', 'PTjxAiXxGeARDcbiaGTX', '319tmp', '0tmp', '23tmp', 'xMSzloXiBwvxCQwrBOHD', '371tmp', 'NDVkvQPRlsXkvUuPvWHH', '81tmp', 'vqPueRbftupYPrphPUWo', 'rpSuECdjmhunVSZKoIwr', 'LdfxBYLvWeJvgayOxBku', 'DGZbCNiwQErOaTIDUvYx', 'EqHVXvXtPeKgwvBlRgLY', 'noIvuQdqEMHFkYinMxTt', 'emWhFfhGTTDwIBoiGHpo', 'rwCedfwYAcWLQtGdukEv', 'vqDdWUOElqQMbgfSyWAv', 'ZdLXrFkRvOYQHqZDniDk', 'GFOodWOtsKdKlCwrvxja', '67tmp', 'qDUnUhKJOIjnKCLVCQKs', 'tyampiqPMDamHWWjBtrs', '192tmp', 'PcbaCokUFHDvYpXNeciD', 'QDgEqZHZrLyVtgrEkMLA', '42tmp', 'ocbsJBzRPZhSHZtfktge', 'KkmVJPaUWQgefCaieGmW', 'PVZNOOrZYAnLWzhcToWX', 'TlMpurJgBJUywTBqsgMF', '289tmp', 'chBVSMgduuAIklZfcZnK', 'qdaqTyoFNcmTfXvSnVcI', 'DMysXzgrEXhwgJtrVeGd', '277tmp', 'ottkFs

correct/total: 15380/20000 76.9%


In [35]:
"""
How much percentage of one group is correctly grouped together (only consider the largest sub-group of grouped ids)
"""

DEBUG = False
real_ids = set(df_walks_connected['real_id'].tolist())
correct, total = 0, 0
for real_id in real_ids:
    id_list = df_walks_connected.loc[df_walks_connected['real_id'] == real_id].sort_values('walk_time')['walker_id'].tolist()
    dic = collections.Counter(id_list)
    if DEBUG: print(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum([v for v in dic.values()])
#     df_walks_connected = df_walks_connected.replace({'walker_id': {key[0]: real_id}})
# print(f'base: {total - (150 / 10 - 1) * 10}/{total} {(total - (150 / 10 - 1) * 10)/total*100}')
# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')

correct/total: 4230/20000 21.15%


In [36]:
cnx.execute('DROP TABLE IF EXISTS walks_attached')
df_walks_connected.to_sql('walks_attached', con=cnx)

## Compute Other Info

In [27]:
id = '1tmp'
path = df_walks_connected.loc[df_walks_connected['walker_id'] == id].sort_values('walk_time')

In [28]:
for (indx1,row1),(indx2,row2) in zip(path[:-1].iterrows(),path[1:].iterrows()):
    jsondict1, jsondict2 = json.loads(row1['json']), json.loads(row2['json'])
    agent_id1, agent_id2 = pos_to_id[(jsondict1['position']['x'], jsondict1['position']['y'])], 
        pos_to_id[(jsondict2['position']['x'], jsondict2['position']['y'])]
    

IndentationError: unexpected indent (<ipython-input-28-e4722ff0a3eb>, line 4)